In [1]:
import numpy as np
import matplotlib.pyplot as plt
import poisson_tools as pt
from scipy.optimize import curve_fit
from scipy.special import expit

In [2]:
def sigmoid_sampling(data, weight, bias):
    sum_data = np.dot(data, weight) + bias
    prob = expit(sum_data)
    rdm = np.random.random(prob.shape)
    index_on = rdm < prob
    samples = np.zeros(prob.shape)
    samples[index_on]=1.
    return samples

def sampling_k(a, b, w, sample_num, init_v):
    gibbs_v = np.zeros((sample_num, a.shape[0]))
    gibbs_h = np.zeros((sample_num, b.shape[0]))
    gibbs_v[0] = init_v
    for g_step in range(1, sample_num):
        gibbs_h[g_step-1] = sigmoid_sampling(gibbs_v[g_step-1], w, b)
        gibbs_v[g_step] = sigmoid_sampling(gibbs_h[g_step-1], w.transpose(), a)
    gibbs_h[-1] = sigmoid_sampling(gibbs_v[-2], w, b)
    return gibbs_v, gibbs_h

def matrix_times(m, n):
    m_matrix = np.transpose(np.tile(m,(len(n), 1)))
    n_matrix = np.tile(n,(len(m), 1))
    return m_matrix*n_matrix

In [3]:
def update_cd1(a, b, W, Data_v):
    max_bsize = Data_v.shape[0]
    for i in range (max_bsize):
        data_h = sigmoid_sampling(Data_v[i], W, b)
        gibbs_v = sigmoid_sampling(data_h, W.transpose(), a)
        gibbs_h = sigmoid_sampling(gibbs_v, W, b)
        delta_a = eta * (Data_v[i] - gibbs_v)
        delta_b = eta * (data_h - gibbs_h)
        delta_w = eta * (matrix_times(Data_v[i], data_h) - matrix_times(gibbs_v, gibbs_h))
        a += delta_a
        b += delta_b
        W += delta_w
    return a, b, W

In [4]:
def update_cdk(a, b, w, data_v, cd_size):
    max_bsize = data_v.shape[0]
    ind_rdm = int(np.floor(np.random.random()*max_bsize))
    init_v = data_v[ind_rdm]
    gibbs_v, gibbs_h = sampling_k(a, b, w, cd_size, init_v)
    avg_h = []
    for vd in range(max_bsize):
        data_h = np.zeros((cd_size, b.shape[0]))
        for h in range(cd_size):
            data_h[h] = sigmoid_sampling(data_v[vd], w, b)
        avg_h.append(np.average(data_h, axis=0))
    
    
    delta_a = eta * (np.average(data_v,0) - np.average(gibbs_v,0))
    delta_b = eta * (np.average(avg_h,0) - np.average(gibbs_h,0))
    pos_delta_w = np.zeros(w.shape)
    neg_delta_w = np.zeros(w.shape)
    for vd in range(max_bsize):
        pos_delta_w += matrix_times(data_v[vd], avg_h[vd])
    for gstep in range(cd_size):
        neg_delta_w += matrix_times(gibbs_v[gstep], gibbs_h[gstep])    
    a += delta_a
    b += delta_b
    w += eta * pos_delta_w/np.float(max_bsize)
    w -= eta * neg_delta_w/np.float(cd_size)
    return a, b, w

In [5]:
def gauss(x, A, mu, sigma):
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

In [79]:
def plot_gibbs(a, b, W, step_num, init_v, title):
    slist_v, slist_h = sampling_k(a, b, W, step_num, init_v)
    gibbs = []
    for i in range(train_num):
        arr = slist_v[i]
        front = np.argmax(arr)
        back = np.argmax(arr[::-1])
        if front + back == 9:
            gibbs.append(front)
        else:
            gibbs.append(-1)
    his = np.histogram(gibbs, bins=np.array(range(digit_num+1))-0.5)
    plt.bar(np.array(range(digit_num))-0.5, his[0], width=np.average(his[1][1:] - his[1][:-1]))
    #parameters, cov_matrix = curve_fit(gauss, np.array(range(digit_num)), his[0])
    #x_plot = np.linspace(-1, 11, 1000)

    #plt.plot(x_plot, gauss(x_plot, *parameters), 'r-', lw=2)
    plt.title(title)
    plt.xlabel('Digits')
    plt.ylabel('Counts')
    plt.ylim((0,300))
    plt.draw()
    plt.savefig('plot/gibbs_%s.pdf'%title)
    plt.show()

In [7]:
train_num = 1000
digit_num = 10
np.random.seed(0)

In [60]:
mu, sigma = 5., 5./3. # mean and standard deviation
s = np.random.normal(mu, sigma, train_num*2)
s = np.floor(s)
ind = np.where((s>=0.) & (s<digit_num) )
train_Data = s[ind[0][:train_num]]

In [8]:
train_Data = np.floor(np.random.rand(1000)*10.)

In [78]:
his = np.histogram(train_Data, bins=np.array(range(digit_num+1))-0.5)
plt.bar(np.array(range(digit_num))-0.5, his[0], width=np.average(his[1][1:] - his[1][:-1]))
parameters, cov_matrix = curve_fit(gauss, np.array(range(digit_num)), his[0])
x_plot = np.linspace(-1, 11, 1000)
plt.plot(x_plot, gauss(x_plot, *parameters), 'g-', lw=2)
plt.ylim((0,300))
plt.title('Training Data Distribution')
plt.xlabel('Digits')
plt.ylabel('Counts')
plt.draw()
plt.savefig('plot/train_data.pdf')
plt.show()


In [62]:
train_Data = train_Data.astype(int)
Data_v = np.zeros((train_num, digit_num))
for i in range(train_num):
    Data_v[i][train_Data[i]] = 1.

In [27]:
hiden_num = digit_num
init_W = np.random.normal(0,0.01,Data_v.shape[1]*hiden_num)
init_W = init_W.reshape((Data_v.shape[1],hiden_num))
init_b = np.zeros(hiden_num)
init_a = np.zeros(digit_num)
#pixel_on = np.sum(Data_v,0)
#init_a = np.log((pixel_on + 0.01)/(train_num - pixel_on + 0.01))

In [37]:
cd_step = 1000
eta = 0.001
a = np.copy(init_a)
b = np.copy(init_b)
W = np.copy(init_W)
for i in range(10000):
    if np.mod(i, 100)==0:
        title = 'cdk_%d'%i
        plot_gibbs(a, b, W, train_num, [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],title )
    a, b, W = update_cdk(a, b, W, Data_v, cd_step)
    

In [84]:
eta = 0.001
a = np.copy(init_a)
b = np.copy(init_b)
W = np.copy(init_W)
for i in range(10):
    title = 'cd1_%d'%(i*train_num)
    plot_gibbs(a, b, W, train_num, [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],title )
    a, b, W = update_cd1(a, b, W, Data_v)

In [53]:
test = range(10)
data_v = Data_v[test]
data_h = sigmoid_sampling(data_v, W, b)
recon = sigmoid_sampling(data_h, W.transpose(), a)
print recon, train_Data[test]

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  1.  0.  0.  0.  0.]] [5 7 6 5 4 6 4 8 9 3]


In [54]:
plot_gibbs(a, b, W, train_num*10, [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.] )

In [55]:
np.save('cd1_10k_a.npy',a)
np.save('cd1_10k_b.npy',b)
np.save('cd1_10k_w.npy',W)

In [58]:
import matplotlib.cm as cm
plt.imshow(W, cmap=cm.gray_r,interpolation='none')
plt.show()